# Thermocouple calculator for ST12

In [1]:
%pylab
%matplotlib tk

from thermocouples_reference import thermocouples
from rdp import rdp
import numpy

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
t = numpy.arange(0, 500, 0.5)

direct = {}
reverse = {}
#for name in thermocouples.keys():
for name in ['C']:
    thermo = thermocouples[name]
    try:
        direct[name] = thermo.emf_mVC(t, Tref=0)
        rvrse = []
        for mv in numpy.arange(max(0, min(direct[name])), max(direct[name]), 0.005):
            rvrse.append((mv, thermo.inverse_CmV(mv, Tref=0)))
        reverse[name] = numpy.array(rvrse)
    except Exception:
        continue

In [46]:
for tr in range(0, 300, 20):
    thermo = thermocouples['C']
    t_cj = 28
    mvr = thermo.emf_mVC(tr, Tref=t_cj) 
    mvCJ = thermo.emf_mVC(t_cj, Tref=0) 
    mv0 = thermo.emf_mVC(tr, Tref=0)
    tcj = thermo.inverse_CmV(mv0+mvCJ, Tref=0)
    u204 = mvr * 1e-3 * 121
    u204_0 = mv0 * 1e-3 * 121
    u204_cj = (mv0+mvCJ) * 1e-3 * 121
    d_u204 = (u204_cj - u204) / 3.3 * 4096
    print(f'{name}: {tr=} {mvr=} {mv0=} {mvCJ=} {u204=} {u204_0=}')

C: tr=0 mvr=-0.38423422770854304 mv0=0.0 mvCJ=0.38423422770854304 u204=-0.04649234155273371 u204_0=0.0
C: tr=20 mvr=-0.11166206728111733 mv0=0.2725721604274257 mvCJ=0.38423422770854304 u204=-0.013511110141015196 u204_0=0.03298123141171851
C: tr=40 mvr=0.1702167110235871 mv0=0.5544509387321301 mvCJ=0.38423422770854304 u204=0.02059622203385404 u204_0=0.06708856358658775
C: tr=60 mvr=0.4609191306885146 mv0=0.8451533583970576 mvCJ=0.38423422770854304 u204=0.055771214813310266 u204_0=0.10226355636604398
C: tr=80 mvr=0.7599756619785114 mv0=1.1442098896870545 mvCJ=0.38423422770854304 u204=0.09195705509939989 u204_0=0.1384493966521336
C: tr=100 mvr=1.0669300320674375 mv0=1.4511642597759806 mvCJ=0.38423422770854304 u204=0.12909853388015993 u204_0=0.17559087543289367
C: tr=120 mvr=1.3813390351652817 mv0=1.7655732628738248 mvCJ=0.38423422770854304 u204=0.1671420232549991 u204_0=0.2136343648077328
C: tr=140 mvr=1.7027723426452723 mv0=2.0870065703538154 mvCJ=0.38423422770854304 u204=0.2060354534600

In [5]:
plt.ylabel('T')
plt.xlabel('mV')
for name in reverse.keys():
    r = reverse[name]
    plt.plot(r[:,0], r[:,1], label=name)

plt.legend()
plt.show()

In [14]:
plt.ylabel('T')
plt.xlabel('mV')
r = reverse['C']
plt.plot(r[:,0], r[:,1], label='C')

plt.legend()
plt.show()

In [18]:
inverse_cmv = numpy.vectorize(thermocouples['C'].inverse_CmV)

plt.ylabel('T')
plt.xlabel('mV')
mv = numpy.arange(0, 8, 0.1)
r0 = inverse_cmv(mv, Tref=0)
r27 = inverse_cmv(mv, Tref=27)
plt.plot(mv, r0, label='C=0')
plt.plot(mv, r0, label='C=27')

plt.legend()
plt.show()

In [21]:
c = numpy.arange(0, 500, 0.5)
direct0 = thermocouples['C'].emf_mVC(c, Tref=0)
direct27 = thermocouples['C'].emf_mVC(c, Tref=27)

plt.ylabel('mV')
plt.xlabel('T')

plt.plot(c, direct0, label='C=0')
plt.plot(c, direct27, label='C=27')

plt.legend()
plt.show()

In [3]:
def u204(mV, R216=120e3, R215=1e3, Vdd=3.3):
    Vu204 = mV * 1e-3 * (R216+R215) / R215
    return int((Vu204 / Vdd) * (1<<12))

u204v = numpy.vectorize(u204)

In [4]:
tsense_adc2C = numpy.copy(reverse['C'])
tsense_adc2C[:,0] = u204v(reverse['C'][:,0])


In [5]:
tsense_adc2C_approx = rdp(tsense_adc2C, epsilon=0.5)
tsense_adc2C_approx

array([[   0.        ,    0.        ],
       [  30.        ,   15.1063702 ],
       [  87.        ,   42.13005025],
       [ 198.        ,   91.83988513],
       [ 343.        ,  152.42194894],
       [ 500.        ,  214.25246872],
       [ 602.        ,  252.95806583],
       [ 801.        ,  325.89215073],
       [1003.        ,  397.56330979],
       [1298.        ,  499.46148306]])

In [14]:
plt.ylabel('T')
plt.xlabel('ADC')
plt.plot(tsense_adc2C[:,0], tsense_adc2C[:,1], label='C')
plt.plot(tsense_adc2C_approx[:,0], tsense_adc2C_approx[:,1], label='Capprox')
plt.legend()
plt.show()

In [34]:
class Thermistor(object):
    def __init__(self, r0=100000, beta=3950, t0=298.15, r_sense=120000):
        self.r0 = r0 # Ohm
        self.beta = beta
        self.t0 = t0 # Kelvin
        self.r_sense = r_sense # Ohm

        self.r_inf = self.r0 * numpy.e ** -(self.beta / self.t0)

    def temp_celsius(self, v_adc):
        """
        Accept ADC value from thermistor connected to ADC_IN0.
        Return temperature in Celsius
        """
        return self.beta / numpy.log(
            self.r_sense / self.r_inf * (1 / (float(0xfff) / v_adc - 1))
        ) - 273.15

    def temp_celsius_v(self, v):
        """
        Accept ADC value from thermistor connected to ADC_IN0.
        Return temperature in Celsius
        """
        return self.beta / numpy.log(
            self.r_sense / self.r_inf * (1 / (3.3/v - 1))
        ) - 273.15
    
    def resistance(self, temp):
        """
        Calculate resistance of a thermistor based on temperature
        """
        return self.r_inf * (numpy.e ** (self.beta/(temp+273.15)))
    
    def temp_from_resistance(self, r):
        return self.beta / numpy.log(r / self.r_inf) - 273.15
    

In [35]:
thermistor = Thermistor()

cj_adc_range = numpy.arange(1, 0xfff, 1)
tC = thermistor.temp_celsius(cj_adc_range)

In [36]:
cj_adc2C = numpy.transpose(numpy.vstack([cj_adc_range, tC]))

In [37]:
cj_adc2C_proper_range = cj_adc2C[[x >= 0 and x <= 100 for x in cj_adc2C[:,1]]]

In [38]:
cj_adc2mV = numpy.copy(cj_adc2C_proper_range)
cj_adc2mV[:,1] = thermocouples['C'].emf_mVC(cj_adc2mV[:,1], Tref=0)

In [39]:
cj_adc2tsense_adc = numpy.copy(cj_adc2mV)
cj_adc2tsense_adc[:, 1] = u204v(cj_adc2tsense_adc[:, 1])
cj_adc2tsense_adc

array([[ 225.,  217.],
       [ 226.,  217.],
       [ 227.,  217.],
       ...,
       [3015.,    0.],
       [3016.,    0.],
       [3017.,    0.]])

In [40]:
cj_adc2tsense_adc_approx = rdp(cj_adc2tsense_adc, epsilon=1)
cj_adc2tsense_adc_approx

array([[ 225.,  217.],
       [ 281.,  198.],
       [ 337.,  183.],
       [ 417.,  166.],
       [ 486.,  154.],
       [ 576.,  141.],
       [ 693.,  127.],
       [ 942.,  104.],
       [1243.,   83.],
       [1602.,   63.],
       [2046.,   42.],
       [3017.,    0.]])

In [4]:
def u402(mV, R406=470e3, R403=20e3, Vdd=3.3):
    Vu402 = mV * 1e-3 * R406 / R403
    return int((Vu402 / Vdd) * (1<<12))

def u402_inverse(adc, R406=470e3, R403=20e3, Vdd=3.3):
    Vu402 = adc * Vdd / (1<<12) 
    return Vu402 * R403 / R406

def current2adc(current):
    return u402(current * 40e-3)

def adc2current(adc):
    return u402_inverse(adc) / 40e-3

u402v = numpy.vectorize(u402)
current2adcv = numpy.vectorize(current2adc)
adc2currentv = numpy.vectorize(adc2current)

In [11]:
adc_range = numpy.arange(0, 0xfff)
adc2current_data = numpy.vstack([adc_range, adc2currentv(adc_range)]).transpose()

In [12]:
adc2current_data

array([[0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 8.57089428e-04],
       [2.00000000e+00, 1.71417886e-03],
       ...,
       [4.09200000e+03, 3.50720994e+00],
       [4.09300000e+03, 3.50806703e+00],
       [4.09400000e+03, 3.50892412e+00]])

In [13]:
plt.plot(adc2current_data[:,0], adc2current_data[:,1])

In [14]:
adc2current_data_approx = rdp(adc2current_data, epsilon=1)
adc2current_data_approx

array([[0.00000000e+00, 0.00000000e+00],
       [4.09400000e+03, 3.50892412e+00]])

In [114]:
Isense = numpy.array([(1.144, 0.9648), (1.25, 1.04), (1.40, 1.1520), (1.6, 1.3131), (1.7, 1.3905), (1.8, 1.4664), (1.9, 1.5415)])

In [132]:
def u402_theoretic(current, R406=470e3, R403=20e3, u201_mV_A=40, Vdd=3.3):
    Vu402 = current * u201_mV_A * 1e-3 * R406 / R403
    return Vu402

u402_theoretic_v = numpy.vectorize(u402_theoretic)

In [130]:
plt.ylabel('Current')
plt.xlabel('Vout')
plt.plot(Isense[:,1], Isense[:,0], label='Actual')
plt.legend()
plt.show()

In [133]:
plt.ylabel('Vout')
plt.xlabel('Current')
plt.plot(Isense[:,0], Isense[:,1], label='Actual')
plt.plot(Isense[:,0], u402_theoretic(Isense[:,0]), label='Theoretic')
plt.legend()
plt.show()

In [134]:
1.465 / 1.284

1.1409657320872275

In [135]:
3509 * 1.141

4003.7690000000002